## Testing RandMatrixMultiply

### Function Definitions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from scipy.sparse import random as sparse_random
# from scipy.sparse import csr_matrix

In [29]:
def randMatrixMultiply(A: np.matrix, B: np.matrix, c: int, P_k: np.array) -> np.matrix:
    """ This function returns an estimator for the product AB using the random matrix multiplication algorithm

    :param A: The first m x n matrix to be multiplied
    :param B: The second n x p matrix to be multiplied
    :param c: The number of column-row pairs to choose
    :param P_k: The probability distribution to choose the probability matrix
    :return: The estimator for the product AB
    """
    n = A.shape[1]
    C = np.zeros((A.shape[0], c)) # m x c
    R = np.zeros((c, B.shape[1])) # c x p
    
    # Optimal Probability Distribution:
    A_col_norms = np.linalg.norm(A, axis=0)  
    B_row_norms = np.linalg.norm(B, axis=1)  
    P_k = (A_col_norms * B_row_norms) / np.sum(A_col_norms * B_row_norms)
    
    sorted_pk = np.flip(np.sort(P_k))
    
    indices = []
    for i in range(len(sorted_pk)):
        index = list(P_k).index(sorted_pk[i])
        indices.append(index)
    
    for t in range(c): # For t = 0 to c-1
        #i_t = np.random.choice(range(n), p=P_k)
        i_t=indices[t]
        coefficient = 1 / (np.sqrt(c * P_k[i_t]))
        C[:, t] = coefficient * A[:, i_t]
        R[t, :] = coefficient * B[i_t, :]
        
    return C @ R

def calculate_accuracy(A: np.matrix, B: np.matrix, c: int, P_k: np.array) -> float:
    """ This function calculates the accuracy of randMatrixMultiply compared to normal matrix multiplication

    :param A: The first m x n matrix to be multiplied
    :param B: The second n x p matrix to be multiplied
    :param c: The number of column-row pairs to choose
    :param P_k: The probability distribution to choose the probability matrix
    :return: The accuracy of the randMatrixMultiply function
    """
    # Compute the product using normal matrix multiplication
    AB_exact = A @ B
    
    # Compute the product using randMatrixMultiply
    AB_approx = randMatrixMultiply(A, B, c, P_k)
    
    # Calculate the Frobenius norm of the difference
    difference = AB_exact - AB_approx
    frobenius_norm = np.linalg.norm(difference, 'fro')
    
    # Calculate the Frobenius norm of the exact product
    frobenius_norm_exact = np.linalg.norm(AB_exact, 'fro')
    
    # Calculate the accuracy as 1 - (norm of difference / norm of exact product)
    accuracy = 1 - (frobenius_norm / frobenius_norm_exact)
    
    return accuracy

def calculate_loss(A: np.matrix, B: np.matrix, c: int, P_k: np.array) -> float:
    """ This function calculates the loss of randMatrixMultiply compared to normal matrix multiplication

    :param A: The first m x n matrix to be multiplied
    :param B: The second n x p matrix to be multiplied
    :param c: The number of column-row pairs to choose
    :param P_k: The probability distribution to choose the probability matrix
    :return: The loss of the randMatrixMultiply function
    """

    AB_exact = A @ B
    AB_approx = randMatrixMultiply(A, B, c, P_k)
    # Calculate the Frobenius norm of the difference
    difference = AB_exact - AB_approx
    frobenius_norm = np.linalg.norm(difference, 'fro')
    
    return frobenius_norm/np.linalg.norm(AB_approx, 'fro')

### Setting parameters

In [35]:
# Set the dimensions of the matrices
m = 10000
n = 1000
p = 500

# # Create the matrices A and B
A = np.random.poisson(size=(m, n))
B = np.random.poisson(size=(n, p))

# Choose the number of column-row pairs to choose
c = 40

### Testing RandMatrixMultiply with uniform probability distribution

In [37]:
# for c in range(1,99):
loss = calculate_loss(A, B, 500, [])
    
print("Loss: "+str(loss))

Loss: 0.053164463714623585


In [ ]:
# Optimal Probability Distribution:
A_col_norms = np.linalg.norm(A, axis=0)  
B_row_norms = np.linalg.norm(B, axis=1)  
P_k = (A_col_norms * B_row_norms) / np.sum(A_col_norms * B_row_norms)

losses = []
for i in range(1000):
    losses.append(calculate_loss(A, B, c, P_k))

print("Mean: "+str(np.mean(losses)))

plt.title('Losses of Multiple trials of RMM algorithm (optimal prob dist)')
plt.xlabel('Losses')
plt.ylabel('Number of trials')
plt.hist(losses)
plt.show()

### Effect of varying number of sampled row/column pairs

In [ ]:
from tqdm import tqdm
#Effect of varying c

A_col_norms = np.linalg.norm(A, axis=0)  
B_row_norms = np.linalg.norm(B, axis=1)  
P_k = (A_col_norms * B_row_norms) / np.sum(A_col_norms * B_row_norms)

averageaccs=[]
for c in tqdm(range(5,100)):
    losses = []
    for i in range(10):
        losses.append(calculate_loss(A, B, c, P_k))
    averageaccs.append(np.mean(losses))

plt.title('Average Loss for different c')
plt.xlabel('c')
plt.ylabel('average loss')
plt.plot(range(5,100), averageaccs)
plt.show()